In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
# from geopy.distance import distance
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [4]:
df1 = pd.read_csv('interventions_ca_geoloc_with_province.csv') 
df2 = pd.read_csv('hospital.csv')
df3 = pd.read_csv('merged_aed_loc_withlatlon.csv')

df_intervention = df1[['lat', 'lon']]
df_hospital = df2[['Latitude', 'Longitude']]
df_aed = df3[['lat','lon']]

In [5]:
df_intervention = df_intervention.dropna()
df_hospital = df_hospital.dropna()
df_aed = df_aed.dropna()

In [6]:
gdf_intervention = gpd.GeoDataFrame(df_intervention, geometry=gpd.points_from_xy(df_intervention.lon,df_intervention.lat),crs="EPSG:4326")
gdf_intervention = gdf_intervention.to_crs(3857) # unit: meter
gdf_hospital = gpd.GeoDataFrame(df_hospital, geometry=gpd.points_from_xy(df_hospital.Longitude, df_hospital.Latitude),crs="EPSG:4326")
gdf_hospital = gdf_hospital.to_crs(3857)
gdf_aed = gpd.GeoDataFrame(df_aed, geometry=gpd.points_from_xy(df_aed.lon, df_aed.lat),crs="EPSG:4326")
gdf_aed = gdf_aed.to_crs(3857)

sindex_hospital = gdf_hospital.sindex
sindex_aed = gdf_aed.sindex

driving_speed = 1000 # driving: 1km/min
walking_speed = 100 # walking: 0.1km/min

df_near = pd.DataFrame()
df_far = pd.DataFrame()
list_near = []
list_far = []

for i, row_intervention in tqdm(gdf_intervention.iterrows(), total=gdf_intervention.shape[0]):
    is_near = False

    possible_hospitals_index = list(sindex_hospital.nearest(row_intervention.geometry, return_all = True, max_distance=4 * driving_speed))
    possible_hospitals_index = [item[0] if item.size else [] for item in possible_hospitals_index]
    if not any(possible_hospitals_index):
            possible_hospitals_index = []
    possible_hospitals = gdf_hospital.iloc[possible_hospitals_index]
    if not possible_hospitals.empty:
      is_near = True

    # possible_hospitals['Distance'] = possible_hospitals.apply(lambda row: distance(row_intervention.geometry.bounds[0:2], row.geometry.bounds[0:2]).km, axis=1)
    # if not possible_hospitals[possible_hospitals['Distance'] <= 4 * driving_speed].empty:
    #     is_near = True

    if not is_near:
        possible_aeds_index = list(sindex_aed.nearest(row_intervention.geometry, return_all = True, max_distance=2 * walking_speed))
        # print(possible_aeds_index)
        possible_aeds_index = [item[0] if item.size else [] for item in possible_aeds_index]
        if not any(possible_aeds_index):
            possible_aeds_index = []

        possible_aeds = gdf_aed.iloc[possible_aeds_index]

        if not possible_aeds.empty:
          is_near = True

        # 计算距离并检查是否有在2分钟内的aed
        # if not possible_aeds.empty:
        #     possible_aeds['Distance'] = possible_aeds.apply(lambda row: distance(row_intervention.geometry.bounds[0:2], row.geometry.bounds[0:2]).km, axis=1)
        #     if not possible_aeds[possible_aeds['Distance'] <= 2 * walking_speed].empty:
        #         is_near = True

    if is_near:
        df_near = df_near.append(df_intervention.loc[i])
        list_near.append(i)
    else:
        df_far = df_far.append(df_intervention.loc[i])
        list_far.append(i)

print(len(df_near))
print(len(df_far))

100%|██████████| 12286/12286 [00:40<00:00, 306.87it/s]

6471
5815


In [7]:
intervention_far = df1.iloc[list_far]
intervention_far.to_csv('intervention_without_aed_hospital.csv', index=False)

In [8]:
intervention_near = df1.iloc[list_near]
intervention_near.to_csv('intervention_within_aed_hospital.csv', index=False)